In [8]:
from keys import * #api tokens, logins: CLIENT_ID, SECRET_KEY, USERNAME, PASSWORD, USER_AGENT

import praw
import pandas as pd
from datetime import datetime

In [2]:
reddit = praw.Reddit(client_id = CLIENT_ID,
                     client_secret = SECRET_KEY,
                     user_agent = USER_AGENT,
                     username = USERNAME,
                     password = PASSWORD
                    ) 

In [3]:
subred = reddit.subreddit("hungary")

In [4]:
new = subred.new(limit = 1000) #search for last 100 posts in "new" section

In [5]:


row_list = []
for post in new:

    try:
        #Post metadata and content
        post_id = post.id
        flair = post.link_flair_text
        author = post.author.name
        timestamp = post.created_utc #unix time, UTC, -2 hours earlier than us
        upvotes = post.score 
        text = post.selftext #it is empty if the text is a link #TODO test for pic and poll
        title = post.title 
        comments = post.comments
        awards = post.all_awardings

        #one post goes to one row in the df
        row = {
            "post_id" : post_id, #str
            "flair" : flair, #str
            "author" : author, #str
            "timestamp" : timestamp, #? str?
            "upvotes" : upvotes, #TODO datatypes
            "text" : text, #str
            "title" : title, #str
            "comments" : comments, # praw.CommentForest
            "awards": awards
        }

        row_list.append(row)

    except AttributeError:
        pass
post_data = pd.DataFrame(row_list)

In [6]:

row_list = []
for _,post_row in post_data.iterrows():

    for comment in post_row.comments:
        try:
            #Comment metadata and content
            comment_id = comment.id
            text = comment.body
            author = comment.author.name
            upvotes = comment.score
            timestamp = comment.created_utc

            #Post metadata
            flair = post_row.flair
            post_id = post_row.post_id
            title = post_row.title
            
            # one row goes to one row in the df
            row = {
                "comment_id" : comment_id, #str
                "text" : text, #str
                "author" : author, #str
                "upvotes" : upvotes, #? str?
                "timestamp" : timestamp, #
                "flair" : flair, #str
                "post_id" : post_id, #str
                "title" : title #str
            }
            row_list.append(row)
        except AttributeError:
            pass

comment_data = pd.DataFrame(row_list)

In [7]:
comment_data

,comment_id,text,author,upvotes,timestamp,flair,post_id,title
0,hfkyz1b,"Elnézést kérek, az előző változatot töröltem, ...",markizaypetermzp,2,1.633509e+09,POLITICS,q2g8eh,"Dobrev: 37%, Márki-Zay: 33%, Karácsony: 24% (A..."
1,hfkxs5h,Lakás,FrostingOtherwise217,24,1.633508e+09,SZÁMOK/ADATOK,q2fz21,Mi az ami már túl nagy luxus a magyar átlagnak?
2,hfkxwd7,"Most mi, vagy az átlag magyar? Erősen felülrep...",Own_Ad4426,8,1.633508e+09,SZÁMOK/ADATOK,q2fz21,Mi az ami már túl nagy luxus a magyar átlagnak?
3,hfkxtmj,A rendszeresen járó kertész / takarító nálam m...,arnoid,12,1.633508e+09,SZÁMOK/ADATOK,q2fz21,Mi az ami már túl nagy luxus a magyar átlagnak?
4,hfkyj7t,"\- friss zöldség, gyümölcs és hús heti többszö...",DavidFromTheHood,10,1.633509e+09,SZÁMOK/ADATOK,q2fz21,Mi az ami már túl nagy luxus a magyar átlagnak?
...,...,...,...,...,...,...,...,...
11028,he7dxym,"Az ""ígéretcunami"" feliratnak a labdán kellene ...",2xar,25,1.632565e+09,POLITICS,pv3auw,Költsük el a GDP tízszeresét is a bérekre!
11029,he7h9v2,"Nem, inkább templomokra, stadionokra, vadászki...",globusporifera,13,1.632568e+09,POLITICS,pv3auw,Költsük el a GDP tízszeresét is a bérekre!
11030,he9nm57,MZP kimutatta a foga fehérjét számomra azzal a...,OJ_drinker,8,1.632605e+09,POLITICS,pv3auw,Költsük el a GDP tízszeresét is a bérekre!
11031,he7ae31,Merjünk kicsik lenni!,ellenkult,-42,1.632562e+09,POLITICS,pv3auw,Költsük el a GDP tízszeresét is a bérekre!


In [14]:
#post_data.to_excel("posts-2021-09-27 00:15:25.xlsx")
comment_data.to_excel("comments.xlsx")

In [9]:
comment_data.timestamp.map(lambda x: datetime.fromtimestamp(x))

0       2021-10-06 10:32:49
1       2021-10-06 10:13:36
2       2021-10-06 10:15:27
3       2021-10-06 10:14:14
4       2021-10-06 10:25:36
                ...        
11028   2021-09-25 12:16:47
11029   2021-09-25 12:59:35
11030   2021-09-25 23:21:29
11031   2021-09-25 11:28:16
11032   2021-09-27 00:15:25
Name: timestamp, Length: 11033, dtype: datetime64[ns]